In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KDTree
import pickle
import warnings
warnings.simplefilter("ignore")

def get_property_info(property_id):
    df = pd.read_csv("recommendation.csv")
    property_info = df.loc[df["propertyid"] == property_id, ["type", "address", "bedroom", "bathroom", "area ( Sq. Yd.)", "price", "purpose"]]
    property_detail = property_info.to_dict("records")[0]
    
    property_type = property_detail.get('type')
    property_address = property_detail.get('address')
    property_purpose = property_detail.get('purpose')
    property_area = property_detail.get('area ( Sq. Yd.)')
    property_bedroom = property_detail.get('bedroom')
    property_bathroom = property_detail.get('bathroom')
    property_price = property_detail.get('price')
    
    prediction = []
    prediction.append(property_address)
    prediction.append(property_type)
    prediction.append(property_purpose)
    prediction.append(property_bedroom)
    prediction.append(property_bathroom)
    prediction.append(property_area)
    prediction.append(property_price)
    
    y = df["image-src"]
    x = df.drop(columns=["location", "propertyid", "description", "mobile", "image-src"])
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
    x_train_df = x_train.copy()
    
    categorical_cols = ["address", "type", "purpose"]
    column_trans = make_column_transformer((OneHotEncoder(sparse=False, drop="first"), 
                                            categorical_cols),(StandardScaler(), 
                                            make_column_selector(dtype_include="number")))

    pipe_new = make_pipeline(column_trans)
    pipe_new.fit(x_train, y_train)
    x_train = pipe_new.fit_transform(x_train)
    
    kdt = KDTree(x_train, leaf_size=30, metric='euclidean')
    x_train_df.to_csv("X_Train.csv")
    
    testing = pd.DataFrame([prediction],columns=["address", "type", "purpose", "bedroom", "bathroom", "area ( Sq. Yd.)", "price"])
    testing = pipe_new.transform(testing)
    indices = kdt.query(testing, k=5, return_distance=False)
    return x_train_df.iloc[indices[0]]

get_property_info(21586854)

,type,address,bedroom,bathroom,area ( Sq. Yd.),purpose,price
328,Flat,"Clifton, Karachi, Sindh",4,5,300,For Rent,185000
11,Flat,"Clifton, Karachi, Sindh",4,5,289,For Rent,185000
16,Flat,"Clifton, Karachi, Sindh",4,5,400,For Rent,220000
13,Flat,"Clifton, Karachi, Sindh",4,4,278,For Rent,160000
237,Flat,"Clifton, Karachi, Sindh",4,4,367,For Rent,260000
